In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
def plt_plot(x,y):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(x,y, c="b")
    plt.show()

In [3]:
data_dire="../01-TrainingData-qLua"
dire_num="01"

In [5]:
df_plc_raw=pd.read_csv(os.path.join(data_dire,dire_num,"PLC/plc.csv"))
print(df_plc_raw.shape,df_plc_raw.columns)
df_plc_raw["time"]=df_plc_raw["time"].str[:5]
mean_spindel=df_plc_raw["spindle_load"].groupby(df_plc_raw['time']).mean()
var_spindel=df_plc_raw["spindle_load"].groupby(df_plc_raw['time']).var()
range_spindel=df_plc_raw["spindle_load"].groupby(df_plc_raw['time']).max()-df_plc_raw["spindle_load"].groupby(df_plc_raw['time']).min()
df_spindel_feature=pd.concat([mean_spindel,var_spindel,range_spindel],axis=1,ignore_index=True)
df_spindel_feature.columns=["mean_spindel","var_spindel","range_spindel"]
df_spindel_feature.index=range(df_spindel_feature.shape[0])
print(df_spindel_feature,df_spindel_feature.columns,df_spindel_feature.shape)
# plt_plot(range(len(range_spindel)),range_spindel)

file_list=os.listdir(os.path.join(data_dire,dire_num,"Sensor"))
c_mean_list=[]
c_abs_mean_list=[]
c_var_list=[]
c_abs_var_list=[]
c_range_list=[]
c_abs_range_list=[]

v_mean_list=[]
v_abs_mean_list=[]
v_var_list=[]
v_abs_var_list=[]
v_range_list=[]
v_abs_range_list=[]
tool_age_list=[]
resudual_life_list=[]
for fileindex in range(1,len(file_list)):
    df_vibration_current=pd.read_csv(os.path.join(data_dire,dire_num,"Sensor",str(fileindex)+'.csv'))
    print(fileindex)
    tool_age_list.append(fileindex*5)
    resudual_life_list.append((len(file_list)-1-fileindex)*5)
    v_mean_list.append(df_vibration_current['vibration_1'].mean())
    v_abs_mean_list.append(abs(df_vibration_current['vibration_1']).mean())
    v_var_list.append(df_vibration_current['vibration_1'].var())
    v_abs_var_list.append(abs(df_vibration_current['vibration_1']).var())
    v_range_list.append(df_vibration_current['vibration_1'].max()-df_vibration_current['vibration_1'].min())
    v_abs_range_list.append(abs(df_vibration_current['vibration_1']).max()-abs(df_vibration_current['vibration_1']).min())
    
    c_mean_list.append(df_vibration_current['current'].mean())
    c_abs_mean_list.append(abs(df_vibration_current['current']).mean())
    c_var_list.append(df_vibration_current['current'].var())
    c_abs_var_list.append(abs(df_vibration_current['current']).var())
    c_range_list.append(df_vibration_current['current'].max()-df_vibration_current['current'].min())
    c_abs_range_list.append(abs(df_vibration_current['current']).max()-abs(df_vibration_current['current']).min())

columns_list=['c_mean','c_abs_mean','c_var','c_abs_var','c_range',
              'c_abs_range','v_mean','v_abs_mean','v_var','v_abs_var',
              'v_range','v_abs_range','tool_age','residual_life']
feature_list=[c_mean_list,c_abs_mean_list,c_var_list,c_abs_var_list,c_range_list,
                c_abs_range_list,v_mean_list,v_abs_mean_list,v_var_list,v_abs_var_list,
                v_range_list,v_abs_range_list,tool_age_list,resudual_life_list]
df_vib_cur_feature=pd.DataFrame(np.array(feature_list).T,columns=columns_list)
print(df_vib_cur_feature.shape)
df_feature=pd.concat([df_spindel_feature,df_vib_cur_feature],axis=1)
print(df_feature.shape,df_feature.columns)
df_feature.to_csv("feature/train_min_"+str(dire_num)+"_feature.csv",index=False)

(110027, 6) Index(['time', 'spindle_load', 'x', 'y', 'z', 'csv_no'], dtype='object')
    mean_spindel  var_spindel  range_spindel
0       0.914081     7.618238      36.545915
1      13.331341    11.455338      19.458602
2      13.701669    11.974730      19.980468
3      13.355145    10.587306      20.871608
4      13.851032    12.947871      21.454512
5      13.781556    19.392014      22.971282
6      14.041365    22.350626      22.760704
7      14.185480    21.674827      22.910245
8      14.179666    21.310683      22.928556
9      14.053676    11.642020      18.958098
10     13.925665    11.979734      20.145268
11     14.273792    19.582601      22.681357
12     14.164259    22.210561      23.398542
13     14.327678    20.250299      23.310038
14     14.251978    21.487087      23.581652
15     14.497593    15.551703      20.358898
16     14.291178    14.314924      22.125919
17     14.261197    11.672504      21.869564
18     14.659725    15.161812      22.125920
19     14.14358

In [ ]:
data_dire="feature"
dire_num="01"

In [ ]:
df_feature = pd.read_csv(os.path.join(data_dire,"train_min_"+str(dire_num)+"_feature.csv"))
print(df_feature.shape,df_feature.columns)
pad_array=np.zeros([10,df_feature.shape[1]])
df_pad=pd.DataFrame(pad_array,columns=df_feature.columns)
df_feature_pad=pd.concat([df_pad,df_feature],axis=0,ignore_index=True)
# print(df_feature_pad.head(15))

df_min5_min10_feature=pd.DataFrame()
for ite in range(11,df_feature_pad.shape[0]+1):
    min5_min10_fea_list=[]
    print(ite)
    mean_list=df_feature_pad[["mean_spindel",'c_abs_mean','v_abs_mean']][ite-5:ite].mean(axis=0)
    min5_min10_fea_list.append(list(mean_list))
    var_list=df_feature_pad[["mean_spindel",'c_abs_mean','v_abs_mean']][ite-5:ite].var(axis=0)
    min5_min10_fea_list.append(list(var_list))
    range_list=df_feature_pad[["mean_spindel",'c_abs_mean','v_abs_mean']][ite-5:ite].max(axis=0)-df_feature_pad[["mean_spindel",'c_abs_mean','v_abs_mean']][ite-5:ite].min(axis=0)
    min5_min10_fea_list.append(list(range_list))
    diff_list=df_feature_pad[["mean_spindel",'c_abs_mean','v_abs_mean']].iloc[ite-1-5]-df_feature_pad[["mean_spindel",'c_abs_mean','v_abs_mean']].iloc[ite-1]
    min5_min10_fea_list.append(list(diff_list))
    
    mean_list=df_feature_pad[["mean_spindel",'c_abs_mean','v_abs_mean']][ite-10:ite].mean(axis=0)
    min5_min10_fea_list.append(list(mean_list))
    var_list=df_feature_pad[["mean_spindel",'c_abs_mean','v_abs_mean']][ite-10:ite].var(axis=0)
    min5_min10_fea_list.append(list(var_list))
    range_list=df_feature_pad[["mean_spindel",'c_abs_mean','v_abs_mean']][ite-10:ite].max(axis=0)-df_feature_pad[["mean_spindel",'c_abs_mean','v_abs_mean']][ite-10:ite].min(axis=0)
    min5_min10_fea_list.append(list(range_list))
    diff_list=df_feature_pad[["mean_spindel",'c_abs_mean','v_abs_mean']].iloc[ite-1-10]-df_feature_pad[["mean_spindel",'c_abs_mean','v_abs_mean']].iloc[ite-1]
    min5_min10_fea_list.append(list(diff_list))
    
    temp_array=np.array(min5_min10_fea_list).reshape([1,-1])
    df_temp=pd.DataFrame(temp_array,columns=["s_mean_min5_mean","c_mean_min5_abs_mean","v_mean_min5_mean",
                                              "s_mean_min5_var","c_mean_min5_abs_var","v_mean_min5_var",
                                              "s_mean_min5_range","c_mean_min5_abs_range","v_mean_min5_range",
                                              "s_mean_min5_diff","c_mean_min5_abs_diff","v_mean_min5_diff",
                                              "s_mean_min10_mean","c_mean_min10_abs_mean","v_mean_min10_mean",
                                              "s_mean_min10_var","c_mean_min10_abs_var","v_mean_min10_var",
                                              "s_mean_min10_range","c_mean_min10_abs_range","v_mean_min10_range",
                                              "s_mean_min10_diff","c_mean_min10_abs_diff","v_mean_min5_diff",
                                              ])
    df_min5_min10_feature=df_min5_min10_feature.append(df_temp,ignore_index=True)
print(df_min5_min10_feature.columns,df_min5_min10_feature.shape)
# print(df_min5_min10_feature.head(5))
df_feature_add_min=pd.concat([df_feature,df_min5_min10_feature],axis=1)
print(df_feature_add_min.columns,df_feature_add_min.shape)   
# df_feature_add_min.to_csv(os.path.join(data_dire,"train_min_"+str(dire_num)+"_feature_add5min.csv"),index=False)

In [3]:
df_fea=pd.read_csv("feature/1_train_addi_feature.csv")
df_fea1=pd.read_csv("feature/1_train_additional_feature.csv")
print(df_fea.head(5),df_fea1.head(5))

   v1_absmean  v2_absmean  v3_absmean  c_absmean  v1_absmedia  v2_absmedia  \
0    0.921373    0.796775    1.354444  32.512001     0.774523     0.654229   
1    1.013382    0.787233    1.459986  32.470704     0.857481     0.648319   
2    1.031298    0.788483    1.333155  32.298331     0.880510     0.651784   
3    1.049083    0.795891    1.351240  32.160430     0.904117     0.658681   
4    1.021497    0.789116    1.353648  32.212419     0.870995     0.652028   

   v3_absmedia  c_absmedia  v1_absrange  v2_absrange      ...        \
0     1.128016   36.357590     0.591384    -1.111593      ...         
1     1.208947   36.451598     0.581259    -2.042686      ...         
2     1.114432   35.997779     0.590042    -0.567012      ...         
3     1.122861   35.963572     0.530752    -0.714348      ...         
4     1.133355   35.817737     0.561373     0.342371      ...         

   v1_minvalue  v2_minvalue  v3_minvalue  c_minvalue  v1_absmin  v2_absmin  \
0    -4.635715    -4.93598